## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


In [2]:
city = 'CDMX'

## Load data

In [3]:
# Download mun_gdf
mun_schema = 'metropolis'
mun_table = 'metro_gdf_2020'

query = f"SELECT * FROM {mun_schema}.{mun_table} WHERE \"city\" LIKE \'{city}\'"
mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(mun_gdf.shape)
mun_gdf.head(2)

(16, 6)


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,city
0,09002,09,002,Azcapotzalco,"POLYGON ((-99.18231 19.50748, -99.18229 19.507...",CDMX
1,09003,09,003,Coyoacán,"POLYGON ((-99.13427 19.35654, -99.13397 19.356...",CDMX


In [ ]:
### Create city area of interest with biggest hexs
big_res = 8
schema_hex = 'hexgrid'
table_hex = f'hexgrid_{big_res}_city_2020'

# Download hexagons with type=urban
type = 'urban'
query = f"SELECT hex_id_{big_res},geometry FROM {schema_hex}.{table_hex} WHERE \"city\" = '{city}\' AND \"type\" = '{type}\'"
hex_urban = aup.gdf_from_query(query, geometry_col='geometry')

In [ ]:
# Downloading NDVI res 10 data
ndvi_schema = 'raster_analysis'
ndvi_table = 'ndvi_analysis_hex'

res = 10

query = f"SELECT * FROM {ndvi_schema}.{ndvi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\" = {res}"
ndvi_res10 = aup.gdf_from_query(query, geometry_col='geometry')

print(ndvi_res10.shape)
ndvi_res10.head(2)

In [4]:
# Downloading NDMI res 10 data
ndmi_schema = 'raster_analysis'
ndmi_table = 'ndmi_analysis_hex'

res = 10

query = f"SELECT * FROM {ndmi_schema}.{ndmi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res}"
ndmi_res10 = aup.gdf_from_query(query, geometry_col='geometry')

print(ndmi_res10.shape)
ndmi_res10.head(2)

(64174, 11)


,hex_id,geometry,res,ndmi_mean,ndmi_std,ndmi_median,ndmi_max,ndmi_min,ndmi_diff,ndmi_tend,city
0,8a49958604d7fff,"POLYGON ((-99.18192 19.23890, -99.18128 19.239...",10,0.052755,0.072827,0.039589,0.159344,-0.041157,0.200502,-0.000828,CDMX
1,8a4995862907fff,"POLYGON ((-99.17475 19.23280, -99.17410 19.233...",10,0.037603,0.058714,0.023568,0.124295,-0.032202,0.156497,-0.000794,CDMX


## Data treatment

### Data treatment - Merge ndvi and ndmi

In [ ]:
ndvi_clean = ndmi_res10.drop(columns=['geometry','res','city'])
ndvi_ndmi_gdf = ndvi_marg.merge(ndvi_clean, left_on='hex_id', right_on='hex_id')

# Show
print(ndvi_ndmi_gdf.shape)
ndvi_ndmi_gdf.head(2)

### Data treatment - Create vegetation categories

In [ ]:
# Create vegetation categories
ndvi_ndmi_gdf['ndvi_median_rango'] = np.nan
ndvi_ndmi_gdf.loc[ndvi_ndmi_gdf.ndvi_median>=0.6 , 'ndvi_median_rango'] = 'Alta densidad vegetal'
ndvi_ndmi_gdf.loc[(ndvi_ndmi_gdf.ndvi_median>=0.4 )&
                  (ndvi_ndmi_gdf.ndvi_median<0.6), 'ndvi_median_rango'] = 'Moderada densidad vegetal'
ndvi_ndmi_gdf.loc[(ndvi_ndmi_gdf.ndvi_median>=0.2)&
                  (ndvi_ndmi_gdf.ndvi_median<0.4), 'ndvi_median_rango'] = 'Mínima densidad vegetal'
ndvi_ndmi_gdf.loc[(ndvi_ndmi_gdf.ndvi_median>=0.1)&
                  (ndvi_ndmi_gdf.ndvi_median<0.2), 'ndvi_median_rango'] = 'Suelo'
ndvi_ndmi_gdf.loc[(ndvi_ndmi_gdf.ndvi_median<0.1), 'ndvi_median_rango'] = 'Suelo artificial/Agua/Piedra'

# Ordering data
categories = ['Suelo artificial/Agua/Piedra', 'Suelo', 'Mínima densidad vegetal', 'Moderada densidad vegetal', 'Alta densidad vegetal']
ndvi_ndmi_gdf['ndvi_median_rango'] = pd.Categorical(ndvi_ndmi_gdf['ndvi_median_rango'], categories=categories, ordered=True)

# Reviewing process
print(ndvi_ndmi_gdf.shape)
ndvi_ndmi_gdf.head(2)

Show ndvi categorical distribution

In [ ]:
ndviproportions = ndvi_ndmi_gdf.groupby('ndvi_median_rango').count()

columns_tokeep = ['hex_id']
ndviproportions = ndviproportions[columns_tokeep]

ndviproportions = ndviproportions.rename(columns={'hex_id':'count'})
ndviproportions['%'] = ndviproportions['count'].apply(lambda x: (x/(ndviproportions['count'].sum()))*100)

ndviproportions

### Data treatment - Filter for places with vegetation

In [ ]:
#Keeping High-moderate-minimal vegetation density
data_withveg = ndvi_ndmi_gdf.loc[(ndvi_ndmi_gdf.ndvi_median_rango == 'Alta densidad vegetal') |
               (ndvi_ndmi_gdf.ndvi_median_rango == 'Moderada densidad vegetal')|
               (ndvi_ndmi_gdf.ndvi_median_rango == 'Mínima densidad vegetal')]

#Reviewing process
print(data_withveg.shape)
data_withveg.head(2)